In [6]:
from dotenv import load_dotenv
load_dotenv()

import os
import enricher
import datetime
import json
import requests

def say_hi():
    print(f"""
    -----------------------
    Welcome to Vesuvius 
    Version Alpha 001 
    -----------------------
    Today is {datetime.date.today()}

    """)

say_hi()


    -----------------------
    Welcome to Vesuvius 
    Version Alpha 001 
    -----------------------
    Today is 2020-04-18

    


## Generate a lat-lng for FourSquare

In [9]:
def geocode(address):
    '''
    Use geocode api to do forward geocoding. https://geocode.xyz/api
    '''
    res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    print(res)
    # Return as GeoJSON -> https://geojson.org/
    return {
        "type":"Point",
        "coordinates": [float(data["longt"]), float(data["latt"])]
    }
geocode('Puerta del Sol')

<Response [200]>


{'type': 'Point', 'coordinates': [21.59814, 48.92664]}

In [53]:
coords = geocode('Puerta del Sol, Madrid, España')
coords

<Response [200]>


{'type': 'Point', 'coordinates': [-3.70343, 40.41703]}

In [54]:
ll = coords
ll = f"{ll['coordinates'][1]},{ll['coordinates'][0]}"
ll

'40.41703,-3.70343'

In [42]:
# I have to make a function to melts Geopoints into 4SQ `ll` format
# This will be the cluster points generated from the crunchbase dataset
#ll ='40.7243,-74.0018' # Times Square
ll

'48.92664,21.59814'

In [55]:
# Define a function that makes multiple categorized requests to the API,
# and receive each request's RESPONSES in an interesting format:
#   OUTPUT will be a DICT
#      - Each key of the output will be one of the keys from the input
#      - The len(array) of Input == len(dict.keys) of Output
#      - The dict.values will be the response generated from the enricher.py module
def cluster_request(cluster):
    results = {}
    for category in cluster:
        results[category] = enricher.getCategoryFrom4SQ(ll,category)
    return results



# IMPORTANT: ♠ This will change later to a list of categories which will be funnelled and output as GeoPoints
# Assign the categories we will look for
flight_points = ['daycare', 'convention center', 'nightlife spot']#'airport', 'heliport', ]

# Call the function
flight_points_cluster = cluster_request(flight_points)

Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: daycare
200
Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: convention center
200
Requesting: https://api.foursquare.com/v2/venues/search,    venue_category: nightlife spot
200


In [56]:
# Exporter.py module

print(" ~ Printing unwinded data")

for key in flight_points:
    print(f" ~ Printing key: {key}")
    response = flight_points_cluster[key]
    
    
    #Export the places to a json
    with open(f'OUTPUT/{key}-raw.json', 'w', encoding='utf-8') as f:
        json.dump(response.json(),f, ensure_ascii=False, indent=4)
               
        
    #print(f"{response.json()}")
    print(f"{type(response)}")
    #print(f" ~ Printing response.json():")
    #print(f"{response.json()}")
       # for venue in response:
        #    print(f" ~ Printing venue: {venue}")
           # print(venue['name'],'\n', venue['location'])


 ~ Printing unwinded data
 ~ Printing key: daycare
<class 'requests.models.Response'>
 ~ Printing key: convention center
<class 'requests.models.Response'>
 ~ Printing key: nightlife spot
<class 'requests.models.Response'>


# Now, I want to turn make the output from the foursquare request include a valid GeoPoint


In [2]:
#print(response.json()['response'].keys())
#print(response.json()['response']['venues'])
#print(response.json()['response']['venues'][0]['location'])
#loc = response.json()['response']['venues'][0]['location']
#response.json()[]

In [58]:
#loc['lng'],loc['lat']
def GeopointFrom4SQ(venue):
    loc = venue['location']
    return {'name':venue['name'],
            'category':venue['categories'][0]['name'], 
            'GeoPoint':{
                'type':'Point',       
                'coordinates':[
                    loc['lng'],
                    loc['lat']
                ]
    }}

GeopointFrom4SQ(response.json()['response']['venues'][0])

{'name': 'Terraza De Gourmet Experiencie Callao',
 'category': 'Beer Garden',
 'GeoPoint': {'type': 'Point', 'coordinates': [-3.7056284, 40.419827]}}

In [59]:
print(response.json().keys())


# Turn the venues into an iterator to be sent to MongoDB
#print(len(venues))
def venues_to_GeoPoints(venues):
    for venue in venues:
        #print(venue)
            # Asigning the values of the GeopointFrom4SQ() function to a new variable
            GeoPoint = GeopointFrom4SQ(venue)

            #print(GeopointFrom4SQ(venue))
            
            yield GeopointFrom4SQ(venue)
        
        
# Access the venues key
#venues = response.json()['response']['venues']
#flight_points_processed = {}

# Call the function, Output will be the array of GeoPoints
#list(venues_to_GeoPoints(venues))

dict_keys(['meta', 'response'])


# Exporter.py module
## Output the Geopoints into `key`.json files

Needs to import pymongo and update the DB automatically


In [60]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost/companies')
db = client.get_database()

In [61]:
list(db.geo.find({},{'name':1}))

[{'_id': ObjectId('5e9b1e69aad39276989eb04a')}]

In [62]:
# Exporter.py module
# Output the Geopoints into `key`.json files

for key in flight_points:
    print(f" ~ Printing key: {key}")
    response = flight_points_cluster[key]
    venues = response.json()['response']['venues']
    print(list(venues_to_GeoPoints(venues)))
    
    #GeoPoint = GeopointFrom4SQ(venue)
    if len(list(venues_to_GeoPoints(venues))) >0:
        db.geo_transports.insert_many(list(venues_to_GeoPoints(venues)))
    #Export the places to a json
    #ith open(f'{key}.json', 'w', encoding='utf-8') as f:c
     #  json.dump(Geopoint,f, ensure_ascii=False, indent=4)
               
        
    #print(f"{response.json()}")
    print(f"{type(response)}")
    
    print(f" ~ Printing response.json():")
    #print(f"{response.json()}")
       # for venue in response:
        #    print(f" ~ Printing venue: {venue}")
           # print(venue['name'],'\n', venue['location'])


 ~ Printing key: daycare
[{'name': 'Guardería Cuatro Pecas', 'category': 'Daycare', 'GeoPoint': {'type': 'Point', 'coordinates': [-3.7175075699415063, 40.4122081465651]}}, {'name': 'Xplora 3D', 'category': "Dentist's Office", 'GeoPoint': {'type': 'Point', 'coordinates': [-3.6948264070026453, 40.40661634455986]}}]
<class 'requests.models.Response'>
 ~ Printing response.json():
 ~ Printing key: convention center
[{'name': "Scherpenseel's", 'category': 'Convention Center', 'GeoPoint': {'type': 'Point', 'coordinates': [-3.710164, 40.416732]}}, {'name': 'eCommerce Made in Spain con Vodafone #eCommerceVF', 'category': 'Convention Center', 'GeoPoint': {'type': 'Point', 'coordinates': [-3.686701917757118, 40.43076377911717]}}, {'name': 'Z.A.R', 'category': 'Meeting Room', 'GeoPoint': {'type': 'Point', 'coordinates': [-3.6955377, 40.4059195]}}, {'name': 'Madrid Konsolosluğu', 'category': 'Convention Center', 'GeoPoint': {'type': 'Point', 'coordinates': [-3.714597702026367, 40.430423736572266]}}

In [63]:
# Access the venues key
venues = response.json()['response']['venues']
flight_points_processed = {}

# Call the function
list(venues_to_GeoPoints(venues))

[{'name': 'Terraza De Gourmet Experiencie Callao',
  'category': 'Beer Garden',
  'GeoPoint': {'type': 'Point', 'coordinates': [-3.7056284, 40.419827]}},
 {'name': 'Perro Malo',
  'category': 'Bar',
  'GeoPoint': {'type': 'Point',
   'coordinates': [-3.7029933929443355, 40.41148649944482]}},
 {'name': 'D Deck',
  'category': 'Cocktail Bar',
  'GeoPoint': {'type': 'Point', 'coordinates': [-3.696989, 40.421752]}},
 {'name': 'Stop Madrid El Leon De Oro',
  'category': 'Wine Bar',
  'GeoPoint': {'type': 'Point', 'coordinates': [-3.6989043, 40.414032]}},
 {'name': 'Cervecería Cruz Blanca',
  'category': 'Bar',
  'GeoPoint': {'type': 'Point',
   'coordinates': [-3.6781005664057997, 40.41693905459416]}}]